#### Вариант №5

||Первый|Второй|Третий|Четвертый|
|---|---|---|---|---|
|Первый|1|2|3|1|
|Второй|0.5|1|3|1|
|Третий|0.33|0.33|1|0.5|
|Четвертый|1|1|2|1|

In [261]:
import numpy as np

In [262]:
matrix = np.array([
    [1, 2, 3, 1],
    [0.5, 1, 3, 1],
    [0.33, 0.33, 1, 0.5],
    [1, 1, 2, 1]
])

normalized_matrix = matrix / matrix.sum(axis=1, keepdims=True)

print("Нормализованная матрица: \n", normalized_matrix)


Нормализованная матрица: 
 [[0.14285714 0.28571429 0.42857143 0.14285714]
 [0.09090909 0.18181818 0.54545455 0.18181818]
 [0.15277778 0.15277778 0.46296296 0.23148148]
 [0.2        0.2        0.4        0.2       ]]


In [263]:
eigenvalues, eigenvectors = np.linalg.eig(matrix)
max_index = np.argmax(eigenvalues)
principal_eigenvector = eigenvectors[:, max_index]
weights = principal_eigenvector / np.sum(principal_eigenvector)
λmax = max(eigenvalues)

print("λmax:", λmax)
print("Собственные значения:", eigenvalues)
print("Собственные векторы:\n", eigenvectors)
print("Главный собственный вектор:", principal_eigenvector)
print("Вектор весов (приоритетов):", weights)

λmax: (4.076676945835413+0j)
Собственные значения: [ 4.07667695+0.j         -0.00691074+0.57590852j -0.00691074-0.57590852j
 -0.06285547+0.j        ]
Собственные векторы:
 [[ 0.68112955+0.j         -0.73125975+0.j         -0.73125975-0.j
  -0.56687091+0.j        ]
 [ 0.47987697+0.j          0.09529148-0.47933042j  0.09529148+0.47933042j
   0.34046658+0.j        ]
 [ 0.20780645+0.j          0.16483358+0.03174468j  0.16483358-0.03174468j
  -0.26062109+0.j        ]
 [ 0.5124423 +0.j          0.0512296 +0.44228809j  0.0512296 -0.44228809j
   0.70343197+0.j        ]]
Главный собственный вектор: [0.68112955+0.j 0.47987697+0.j 0.20780645+0.j 0.5124423 +0.j]
Вектор весов (приоритетов): [0.36206121+0.j 0.25508339+0.j 0.11046159+0.j 0.27239382+0.j]


##### EM

In [264]:
eigenvalues, eigenvectors = np.linalg.eig(matrix.T)
max_eigenvalue_index = np.argmax(eigenvalues)
weights = np.real_if_close(eigenvectors[:, max_eigenvalue_index].T / np.sum(eigenvectors[:, max_eigenvalue_index]))

print("Веса каналов (метод ЕМ):", weights)

Веса каналов (метод ЕМ): [0.14416284 0.20462238 0.46266194 0.18855284]


##### RGMM

In [265]:
geometric_means = np.power(np.prod(matrix, axis=1), 1/len(matrix))
weights_rgmm = geometric_means / np.sum(geometric_means)

print("Веса каналов (метод RGMM):", weights_rgmm)


Веса каналов (метод RGMM): [0.36028387 0.25475916 0.11120043 0.27375654]


##### AN

In [266]:
eigenvalues_an, eigenvectors_an = np.linalg.eig(matrix.T)
max_eigenvalue_index_an = np.argmax(eigenvalues_an)
weights_an = np.real_if_close(eigenvectors_an[:, max_eigenvalue_index_an].T / np.sum(eigenvectors_an[:, max_eigenvalue_index_an]))

print("Веса каналов (метод AN):", weights_an)

Веса каналов (метод AN): [0.14416284 0.20462238 0.46266194 0.18855284]


##### Линия

In [267]:
preference_values = {
    'A': 0.5,
    'C': 3,
    'D': 1,
}

weight_b = 1 / sum(preference_values.values())

weights_line_method = {}
for alternative in preference_values:
    weight = preference_values[alternative] * weight_b
    weights_line_method[alternative] = weight

total_weight = sum(weights_line_method.values())
normalized_weights_line_method = {alternative: weight / total_weight for alternative, weight in weights_line_method.items()}

normalized_weights_line_method['B'] = weight_b

print("Веса каналов (метод 'линия'):", normalized_weights_line_method)


Веса каналов (метод 'линия'): {'A': 0.11111111111111112, 'C': 0.6666666666666667, 'D': 0.22222222222222224, 'B': 0.2222222222222222}


##### MRCI

In [268]:
def calculate_CI(matrix):
    eigenvalues, _ = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    max_eigenvalue = eigenvalues[max_index].real
    n = len(matrix)
    CI = (max_eigenvalue - n) / (n - 1)
    return CI

num_matrices = 10
matrix_size = 4

CI_values = []

for _ in range(num_matrices):
    random_matrix = np.random.randint(1, 10, size=(matrix_size, matrix_size))
    CI = calculate_CI(random_matrix)
    CI_values.append(CI)

MRCI = np.mean(CI_values)

print("Индексы согласованности для сгенерированных матриц:")
for i, CI in enumerate(CI_values, 1):
    print(f"Матрица {i}: CI = {CI}")

print(f"\nСреднее значение индексов согласованности (MRCI): {MRCI}")


Индексы согласованности для сгенерированных матриц:
Матрица 1: CI = 5.540551565980565
Матрица 2: CI = 5.586124760703878
Матрица 3: CI = 4.20238215268143
Матрица 4: CI = 6.311248864480748
Матрица 5: CI = 5.167219461586062
Матрица 6: CI = 4.929775831647606
Матрица 7: CI = 5.717994330707232
Матрица 8: CI = 4.0549153692473885
Матрица 9: CI = 6.155071926893867
Матрица 10: CI = 4.869914836659809

Среднее значение индексов согласованности (MRCI): 5.253519910058859


$$
CI = \frac{λ_{max} - n}{n - 1}
$$

In [269]:
CI = (λmax - len(matrix)) / (len(matrix) - 1)
print("CI:", CI)

CI: (0.02555898194513754+0j)


$$
CR = \frac{CI}{MRCI}
$$

In [270]:
CR = CI/MRCI

print("Отношение согласованности CR:", CR)

Отношение согласованности CR: (0.004865115652498057+0j)


$$
GCI = \frac{λ_{max} - n}{n - 1}
$$

In [271]:
GCI = (λmax - len(matrix)) / (len(matrix) - 1)
print("Геометрический индекс согласованности GCI:", GCI)

Геометрический индекс согласованности GCI: (0.02555898194513754+0j)


##### HM(s)

In [272]:
def calculate_hm(row):
    n = len(row)
    harmonic_sum = sum(1 / element for element in row)
    hm = n / harmonic_sum
    return hm

HMs = [calculate_hm(row) for row in normalized_matrix]

for i, hm in enumerate(HMs):
    print(f"Гармоническое среднее для строки {i + 1}: {hm}")

Гармоническое среднее для строки 1: 0.20168067226890754
Гармоническое среднее для строки 2: 0.16783216783216784
Гармоническое среднее для строки 3: 0.20438498699368265
Гармоническое среднее для строки 4: 0.22857142857142856


$$
HCI(n) = \frac{(HM(s) - n)(n + 1)}{n(n-1)}
$$

In [273]:
n = matrix.shape[0]
RI = {3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49}[n]
HCI = CI / RI

print("Гармонический индекс согласованности (HCI):", HCI)

Гармонический индекс согласованности (HCI): (0.0283988688279306+0j)


##### HRCI(n)

$$
HCR(n)=\frac{HCI(n)}{HRCI(n)}
$$

In [274]:
HCR = HCI / 0.89
print("Гармонические отношения согласованности HCR:", HCR)

Гармонические отношения согласованности HCR: (0.03190884137969731+0j)


$$
\begin{align*}
k_c &= \min_{h=1,\dots,5} k_c(R_h) 
\end{align*}
$$

In [275]:
def calculate_spectrum(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    max_index = np.argmax(eigenvalues)
    principal_eigenvector = eigenvectors[:, max_index]
    return principal_eigenvector / np.sum(principal_eigenvector)

n_matrices = 5

spectra = []
for _ in range(n_matrices):
    random_matrix = np.random.rand(normalized_matrix.shape[0], normalized_matrix.shape[1])
    random_matrix /= random_matrix.sum(axis=1)[:, np.newaxis]
    spectrum = calculate_spectrum(random_matrix)
    spectra.append(spectrum)

kc_values = [np.min(kc) for kc in spectra]
kc = min(kc_values)

print("Спектральный коэффициент согласованности (kc):", kc)

Спектральный коэффициент согласованности (kc): (0.2499999999999999-0j)


In [276]:
matrix = np.array([
    [1, 0.5, 0.25, 2, 1, 0.5, 2],
    [2, 1, 0.25, 0.5, 1, 0.5, 2],
    [4, 4, 1, 2, 5, 2, 0.142],
    [0.5, 2, 0.5, 1, 2, 1, 4],
    [1, 1, 0.2, 0.5, 1, 0.5, 2],
    [2, 2, 0.5, 1, 2, 1, 4],
    [0.5, 0.5, 7, 0.25, 0.5, 0.25, 1]])

normalized_matrix = matrix / matrix.sum(axis=1, keepdims=True)
print("Нормализованная матрица:\n", normalized_matrix)

Нормализованная матрица:
 [[0.13793103 0.06896552 0.03448276 0.27586207 0.13793103 0.06896552
  0.27586207]
 [0.27586207 0.13793103 0.03448276 0.06896552 0.13793103 0.06896552
  0.27586207]
 [0.22048286 0.22048286 0.05512071 0.11024143 0.27560357 0.11024143
  0.00782714]
 [0.04545455 0.18181818 0.04545455 0.09090909 0.18181818 0.09090909
  0.36363636]
 [0.16129032 0.16129032 0.03225806 0.08064516 0.16129032 0.08064516
  0.32258065]
 [0.16       0.16       0.04       0.08       0.16       0.08
  0.32      ]
 [0.05       0.05       0.7        0.025      0.05       0.025
  0.1       ]]


In [277]:
eigenvalues, _ = np.linalg.eig(matrix)
λmax = max(eigenvalues)
n = len(matrix)
threshold = 0.1
CI = (λmax - n)/(n - 1)
print(f"CI = {CI}")
CR = CI / 1.32
print(f"CR = {CR}")
if CR > threshold:
    print("МПС несогласована!")
    

CIs = []
for i in range(len(matrix)):
    m_copy = np.copy(matrix)
    eigenvalues, _ = np.linalg.eig(np.delete(np.delete(matrix, i, axis=0), i, axis=1))
    λmax = max(eigenvalues)
    n = len(matrix)
    CIs.append(((λmax - n)/(n - 1), i))
    print(f"{i + 1} CI = {CIs[i][0]}")

min_CIs = sorted(CIs)[:2]
print(f"Выброс: [{min_CIs[0][1] + 1}, {min_CIs[1][1] + 1}] = {matrix[min_CIs[1][1]][min_CIs[0][1]]}")


CI = (0.5230140224026594+0j)
CR = (0.39622274424443893+0j)
МПС несогласована!
1 CI = (0.3172765558444972+0j)
2 CI = (0.3490111768319502+0j)
3 CI = (-0.10735911872128619+0j)
4 CI = (0.32458515077366956+0j)
5 CI = (0.34508108797220505+0j)
6 CI = (0.362099097740607+0j)
7 CI = (-0.10649746567621943+0j)
Выброс: [3, 7] = 7.0


In [278]:
row_correlations = np.corrcoef(normalized_matrix)
column_correlations = np.corrcoef(normalized_matrix, rowvar=False)
min_row_corr_value = np.min(row_correlations[0, 1:])
min_row_corr_index = np.argmin(row_correlations[0, 1:]) + 2
min_column_corr_value = np.min(column_correlations[0, 1:])
min_column_corr_index = np.argmin(column_correlations[0, 1:]) + 2

print("Минимальный коэффициент корреляции для строки:", min_row_corr_value)
print("Индекс строки с минимальным коэффициентом корреляции:", min_row_corr_index)
print("Минимальный коэффициент корреляции для столбца:", min_column_corr_value)
print("Индекс столбца с минимальным коэффициентом корреляции:", min_column_corr_index)

Минимальный коэффициент корреляции для строки: -0.44768942874169837
Индекс строки с минимальным коэффициентом корреляции: 7
Минимальный коэффициент корреляции для столбца: -0.5288400366285384
Индекс столбца с минимальным коэффициентом корреляции: 3


In [279]:
D = normalized_matrix
def build_empirical_matrix(arr):
    n_rows, n_cols = arr.shape
    T = np.zeros((n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            T[i, j] = np.sum(arr[i, :] > arr[i, j])

    return T

T = build_empirical_matrix(D)
print("\nЭмпирическая матрица T:")
print(T)

deviation_matrix = T - D

print("Матрица отклонений:")
print(deviation_matrix)

n = D.shape[0] * D.shape[1]
mean_delta = np.sum(D) / n**2

var_delta = np.sum((D - mean_delta)**2) / (n**2 - 1)

std_dev_delta = np.sqrt(var_delta)

print(f"Математическое ожидание M(∆) = {mean_delta}")
print(f"Дисперсия Var(∆) = {var_delta}")
print(f"Стандартное отклонение StdDev(∆) = {std_dev_delta}")



from scipy.stats import chi2

def detect_outliers_matrix(data):
    n_samples, n_dimensions = data.shape
    
    flattened_data = data.flatten()
    mean = np.mean(flattened_data)
    std_dev = np.std(flattened_data)
    
    chi2_crit = chi2.ppf(-0.00001,  df=n_samples * n_dimensions - 1)
    
    outliers = []
    for value in flattened_data:
        z_score = (value - mean) / std_dev
        chi2_val = z_score ** 2
        
        if chi2_val > chi2_crit:
            outliers.append(value)
    
    return outliers

# Пример использования:
data = matrix  # Многомерный массив данных
outliers = detect_outliers_matrix(data)
print("Выбросы:", outliers)



Эмпирическая матрица T:
[[2. 4. 6. 0. 2. 4. 0.]
 [0. 2. 6. 4. 2. 4. 0.]
 [1. 1. 5. 3. 0. 3. 6.]
 [5. 1. 5. 3. 1. 3. 0.]
 [1. 1. 6. 4. 1. 4. 0.]
 [1. 1. 6. 4. 1. 4. 0.]
 [2. 2. 0. 5. 2. 5. 1.]]
Матрица отклонений:
[[ 1.86206897  3.93103448  5.96551724 -0.27586207  1.86206897  3.93103448
  -0.27586207]
 [-0.27586207  1.86206897  5.96551724  3.93103448  1.86206897  3.93103448
  -0.27586207]
 [ 0.77951714  0.77951714  4.94487929  2.88975857 -0.27560357  2.88975857
   5.99217286]
 [ 4.95454545  0.81818182  4.95454545  2.90909091  0.81818182  2.90909091
  -0.36363636]
 [ 0.83870968  0.83870968  5.96774194  3.91935484  0.83870968  3.91935484
  -0.32258065]
 [ 0.84        0.84        5.96        3.92        0.84        3.92
  -0.32      ]
 [ 1.95        1.95       -0.7         4.975       1.95        4.975
   0.9       ]]
Математическое ожидание M(∆) = 0.0029154518950437317
Дисперсия Var(∆) = 0.0006998330767409577
Стандартное отклонение StdDev(∆) = 0.026454358369481536
Выбросы: []


In [283]:

scores_matrix = normalized_matrix

alphas = [0.5, 0.6, 0.7, 0.8, 0.9]

def compute_wgmm(scores, alpha):
    wgmm_result = np.prod(scores ** alpha, axis=1) ** (1 / scores.shape[1])
    return wgmm_result

def compute_wamm(scores, alpha):
    wamm_result = np.mean(scores ** alpha, axis=1)
    return wamm_result

results = []
for alpha in alphas:
    wgmm = compute_wgmm(scores_matrix, alpha)
    wamm = compute_wamm(scores_matrix, alpha)
    consistency_ratio = np.abs(wgmm - wamm)
    weights = np.abs(wgmm - wamm) / np.max(np.abs(wgmm - wamm))
    deviation = np.mean(consistency_ratio)
    std_deviation = np.std(consistency_ratio)
    results.append((alpha, wgmm, wamm, consistency_ratio, weights, deviation, std_deviation))

columns = ["α", "WGMM", "WAMM", "Consistency Ratio", "Weights", "Deviation", "Std Deviation"]
import pandas as pd
df = pd.DataFrame(results, columns=columns)

df


,α,WGMM,WAMM,Consistency Ratio,Weights,Deviation,Std Deviation
0,0.5,"[0.33637732426088945, 0.33637732426088945, 0.3...","[0.35773626931456143, 0.35773626931456143, 0.3...","[0.021358945053671974, 0.021358945053671974, 0...","[0.43808884803361425, 0.43808884803361425, 0.8...",0.027378,0.011600
1,0.6,"[0.2705154374487822, 0.2705154374487822, 0.243...","[0.2953431181015082, 0.29534311810150815, 0.29...","[0.024827680652726003, 0.024827680652725947, 0...","[0.43044502373182825, 0.43044502373182725, 0.8...",0.031703,0.013380
2,0.7,"[0.21754915275248968, 0.21754915275248968, 0.1...","[0.2448676551476235, 0.2448676551476235, 0.241...","[0.02731850239513381, 0.02731850239513381, 0.0...","[0.4213820046144948, 0.4213820046144948, 0.757...",0.034821,0.014776
3,0.8,"[0.1749535416894158, 0.1749535416894158, 0.152...","[0.20383995365985103, 0.203839953659851, 0.202...","[0.028886411970435233, 0.028886411970435205, 0...","[0.4109856182101731, 0.4109856182101727, 0.710...",0.036827,0.015856
4,0.9,"[0.14069805081932166, 0.14069805081932166, 0.1...","[0.17033744949395604, 0.17033744949395604, 0.1...","[0.029639398674634387, 0.029639398674634387, 0...","[0.3993643152660942, 0.3993643152660942, 0.665...",0.037871,0.016685


In [282]:
def calculate_weights_triangle(comparison_matrix):
    n = comparison_matrix.shape[0]
    weights = np.zeros(n)
    
    for i in range(n):
        ae_index = i
        temp_matrix = np.delete(np.delete(comparison_matrix, ae_index, axis=0), ae_index, axis=1)
        weights_temp = np.sum(temp_matrix, axis=1) / (n - 1)
        weights[i] = np.mean(weights_temp)
    return weights


weights_triangle = calculate_weights_triangle(normalized_matrix)
print("Согласованные веса по методу треугольника:", weights_triangle)

Согласованные веса по методу треугольника: [0.14130306 0.14326231 0.14203672 0.14886905 0.14046434 0.15431315
 0.1231731 ]
